In [1]:
import csv
import pandas as pd
df=pd.DataFrame(columns=["Tweet_index","Label","Tweet_Text"]);
txt_file=r"SemEval2018-T3-train-taskB_emoji_ironyHashtags.txt"
#csv_file=r"data11.csv"
i=0
with open(txt_file,encoding="utf-8") as f:
    for line in f:
        #print(line)
        if i==0:
            i=i+1
            continue
        df.loc[i]=[text for text in line.split("\t")]
        i=i+1
df.to_csv('input_data_TrainB.csv', sep=' ',index=False)
tweets=df["Tweet_Text"]
label= df["Label"]
print(len(tweets))
print(len(label))


3834
3834


In [2]:
import emoji
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
input_txt=""
non_normalized_input_txt=""
for line in tweets:
    
    #print(line)
    non_normalized_input_txt+=line
    de_emojize=emoji.demojize(line)
    remove_multidots = re.sub('\.\.\.+','...',de_emojize)
    remove_verticalbars = re.sub(r'\|', '',remove_multidots)
    remove_multiplewhite=re.sub(' +',' ',remove_verticalbars)
    commonurl_str = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', 'http://someurl', remove_multiplewhite)
    replace_username=re.sub('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','@someuser',commonurl_str)
  #  puncutation_flooding=re.sub(r'['+string.punctuation+']+', ' ',replace_username)
    input_txt+=replace_username
#print(input_txt)
stop_words = set(stopwords.words('english'))
input_tokenized=nltk.word_tokenize(input_txt)
filtered_sentence = [w for w in input_tokenized if not w in stop_words]
#print(input_tokenized)
input_posTag=nltk.pos_tag(filtered_sentence)
#print(input_posTag)

In [3]:
def punct_flooding(str):
    l = len(str)
    count=0
    for i in range(l):
        cur_count=1
        if str[i] in string.punctuation:
            for j in range(i+1,l):
                if str[j] not in string.punctuation or str[j] in ['/','#','.']:
                    break
                else:
                    cur_count+=1
        if cur_count > count: 
            count = cur_count 
    if count >2:
        return 1
    else:
        return 0

In [4]:
def char_flooding(str):
    l = len(str)
    count=0
    for i in range(l):
        cur_count=1
        if str[i].isalpha and str[i] not in string.punctuation and not str[i].isnumeric :
             for j in range(i+1,l):
                if not str[j] == str[i] :
                    break
                else:
                    cur_count+=1
        if cur_count > count: 
            count = cur_count 
    if count >2:
        return 1
    else:
        return 0

In [5]:
def punct_present(str):
    l = len(str)
    cur_count=0
    for i in range(l):
        if str[i] in string.punctuation and str[i] not in ['/','#','.'] :
            cur_count=1
            break;
    return cur_count

In [6]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
import pandas as pd
import re
import emoji
lexfeatures=[]
i=0
for line in input_txt.split('\n'):
    if(len(line)>0):
        hashCount=line.count("#")
        ispunctflooded= punct_flooding(line)
        ispunctpresent= punct_present(line)
        #print(ispunctpresent)
        ischarflooded=char_flooding(line)
        emojislist=re.findall(r':[\w]+:',line)
        emojiCount=len(emojislist)
        tweetLength=len(line)
        wordsList= line.split(' ')
        numberOfWords=len(wordsList)
        hashtagWordRatio= round(hashCount/numberOfWords,2)
        #print(emojislist)
        #print(emojiCount)
        lexfeatures.insert(i,{'Punctuation': ispunctpresent,'Character_Flooding':ischarflooded,'Punctutation_Flooding':ispunctflooded,'Hashtag_Frequency':hashCount,'Hashtag_to_word_ratio':hashtagWordRatio,'Emoticon_Count':emojiCount,'Tweet_length':tweetLength})
        i=i+1
df = pd.DataFrame(lexfeatures)
df.to_csv('outfiles/lexFeatures_TrainB.csv', sep='\t',index=False)
#print(df)

In [8]:
def getCount(dict11):
    dict13={}
    for key, value in dict11.items():
        if value>=2:
            value=2
        dict13[key]=value
    return dict13

In [9]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
def get_continuous_chunks(text):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    return continuous_chunk

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def ispolarityClash(line):
    pos=0
    neg=0
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(line)
    for k in sorted(ss):
        #print('{0}: {1}, '.format(k, ss[k]), end='')
        #print()
        if k=='neg':
            neg=ss[k]
        if k=='pos':
            pos=ss[k]
        if(pos >0 and neg >0):
            return 1
        else:
            return 0

/home/sharath/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [11]:
def istenseclash(sentence):
    print(sentence)
    text = word_tokenize(sentence)
    tagged = pos_tag(text)
    clash=0
    tense = {}
    tense["future"] = len([word for word in tagged if word[1] == "MD"])
    tense["present"] = len([word for word in tagged if word[1] in ["VBP", "VBZ","VBG"]])
    tense["past"] = len([word for word in tagged if word[1] in ["VBD", "VBN"]]) 
    if tense["past"]>0  and tense["present"]>0:
        clash=1
    print(tense)
    return clash

In [12]:
print(istenseclash("I hate that  when  you get a girlfriend  most  of  the  girls  that didnt want  you  all  of  a  sudden want you"))

I hate that  when  you get a girlfriend  most  of  the  girls  that didnt want  you  all  of  a  sudden want you
{'future': 0, 'present': 4, 'past': 0}
0


In [13]:
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk.tokenize import word_tokenize
from collections import Counter
i=CC_count=0
#count11=[]
syntacticfeatures_count=[]
syntacticfeatures_presence=[]
syntacticfeatures_greater2=[]
syntacticfeatures_ner=[]
for line in input_txt.split('\n'):
    if(len(line)>0):
        count= Counter([j for i,j in pos_tag(word_tokenize(line))])
        #test12=ne_chunk(pos_tag(word_tokenize(line)))
        #print(test12)
        list11=sorted(count.most_common())
        is_ner_present=0
        length_of_ner= len(get_continuous_chunks(line))
        if length_of_ner >0:
            is_ner_present=1
        frequency_ner=length_of_ner/len(line)
        dict11=dict(list11)
        dict13=getCount(dict11)
        dict12=dict.fromkeys(dict11,1)
        tense_clash=istenseclash(line)
        print(tense_clash)
        #count11=getabsoluteCount(dict11,count11)
        syntacticfeatures_count.insert(i,{'CC':dict11.get('CC',0),'JJ':dict11.get('JJ',0),'JJR':dict11.get('JJR',0),'JJS':dict11.get('JJS',0),'MD':dict11.get('MD',0),'NN':dict11.get('NN',0),'NNP':dict11.get('NNP',0),'NNS':dict11.get('NNS',0),'NNPS':dict11.get('NNPS',0),'PRP':dict11.get('PRP',0),'POS':dict11.get('POS',0),'RB':dict11.get('RB',0),'PRP$':dict11.get('PRP$',0),'RBR':dict11.get('RBR',0),'RBS':dict11.get('RBS',0),'UH':dict11.get('UH',0),'VB':dict11.get('VB',0),'VBD':dict11.get('VBD',0),'VBG':dict11.get('VBG',0),'VBP':dict11.get('VBP',0),'VBN':dict11.get('VBN',0),'VBZ':dict11.get('VBZ',0),'WDT':dict11.get('WDT',0),'WP':dict11.get('WP',0),'WP$':dict11.get('WP$',0),'WRB':dict11.get('WRB',0)})
        syntacticfeatures_presence.insert(i,{'CC':dict12.get('CC',0),'JJ':dict12.get('JJ',0),'JJR':dict12.get('JJR',0),'JJS':dict12.get('JJS',0),'MD':dict12.get('MD',0),'NN':dict12.get('NN',0),'NNP':dict12.get('NNP',0),'NNS':dict12.get('NNS',0),'NNPS':dict12.get('NNPS',0),'PRP':dict12.get('PRP',0),'POS':dict12.get('POS',0),'RB':dict12.get('RB',0),'PRP$':dict12.get('PRP$',0),'RBR':dict12.get('RBR',0),'RBS':dict12.get('RBS',0),'UH':dict12.get('UH',0),'VB':dict12.get('VB',0),'VBD':dict12.get('VBD',0),'VBG':dict12.get('VBG',0),'VBP':dict12.get('VBP',0),'VBN':dict12.get('VBN',0),'VBZ':dict12.get('VBZ',0),'WDT':dict12.get('WDT',0),'WP':dict12.get('WP',0),'WP$':dict12.get('WP$',0),'WRB':dict12.get('WRB',0)})
        syntacticfeatures_greater2.insert(i,{'CC':dict13.get('CC',0),'JJ':dict13.get('JJ',0),'JJR':dict13.get('JJR',0),'JJS':dict13.get('JJS',0),'MD':dict13.get('MD',0),'NN':dict13.get('NN',0),'NNP':dict13.get('NNP',0),'NNS':dict13.get('NNS',0),'NNPS':dict13.get('NNPS',0),'PRP':dict13.get('PRP',0),'POS':dict13.get('POS',0),'RB':dict13.get('RB',0),'PRP$':dict13.get('PRP$',0),'RBR':dict13.get('RBR',0),'RBS':dict13.get('RBS',0),'UH':dict13.get('UH',0),'VB':dict13.get('VB',0),'VBD':dict13.get('VBD',0),'VBG':dict13.get('VBG',0),'VBP':dict13.get('VBP',0),'VBN':dict13.get('VBN',0),'VBZ':dict13.get('VBZ',0),'WDT':dict13.get('WDT',0),'WP':dict13.get('WP',0),'WP$':dict13.get('WP$',0),'WRB':dict13.get('WRB',0)})
        syntacticfeatures_ner.insert(i,{'NER_Presence':is_ner_present,'NER_Count':length_of_ner,'Frequency_of_NER':frequency_ner,'Tense_Clash':tense_clash})
        i=i+1
df = pd.DataFrame(syntacticfeatures_count)
df1=pd.DataFrame(syntacticfeatures_presence)
df2=pd.DataFrame(syntacticfeatures_greater2)
df3=pd.DataFrame(syntacticfeatures_ner)
df.to_csv('outfiles/synFeatures_countofTagPerTweet_TrainB.csv', sep='\t',index=False)
df1.to_csv('outfiles/synFeatures_presenceTaginTweet_TrainB.csv', sep='\t',index=False)
df2.to_csv('outfiles/synFeatures_greaterthan2inTweet_TrainB.csv', sep='\t',index=False)
df3.to_csv('outfiles/synFeatures_NamedEntityRecognition_TrainB.csv', sep='\t',index=False)
#list11=sorted(count.iteritems())
#print(list11)

Sweet United Nations video. Just in time for Christmas. #imagine #NoReligion #irony http://someurl
{'future': 0, 'present': 0, 'past': 0}
0
@someuser We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing #Sarcasm ;)
{'future': 0, 'present': 2, 'past': 3}
1
Hey there! Nice to see you Minnesota/ND Winter Weather #Not
{'future': 0, 'present': 0, 'past': 0}
0
3 episodes left I'm dying over here
{'future': 0, 'present': 2, 'past': 1}
1
"I can't breathe!" was chosen as the most notable quote of the year in an annual list released by a Yale University librarian #irony
{'future': 1, 'present': 0, 'past': 3}
0
You're never too old for Footie Pajamas. http://someurl
{'future': 0, 'present': 1, 'past': 0}
0
Nothing makes me happier then getting on the highway and seeing break lights light up like a Christmas tree.. #not
{'future': 0, 'present': 3, 'past': 1}
1
4:30 an opening my first beer now gonna be a long night/day
{'future': 0, 'presen

@someuser http://someurl #CHRISTMAS #GIFT GET STARTED AT MAKING MONEY IN SOCIAL MEDIA @ http://someurl
{'future': 0, 'present': 0, 'past': 1}
0
don't knw how to react everytime sum1 says me being in a wheelchair "inspires" them! im just living they same you are dude
{'future': 0, 'present': 5, 'past': 0}
0
GET AHT A TAHN!The Made in PGH Gift Guide http://someurl via @someuser
{'future': 0, 'present': 0, 'past': 0}
0
On my lunch break so sleepy:sleeping_face:
{'future': 0, 'present': 0, 'past': 0}
0
#Work#Trains#TheGrind#London#Business#MyHappyFace#Not http://someurl
{'future': 0, 'present': 0, 'past': 0}
0
dude said I better think hard , so he can take me to cheddars lmao ! #not
{'future': 1, 'present': 0, 'past': 1}
0
My #Christmas ritual...part one. #HowTheGrinchStoleChristmas http://someurl
{'future': 0, 'present': 0, 'past': 0}
0
Overcast at Creso. #todayimloving #beachwalk #crescenthead http://someurl
{'future': 0, 'present': 1, 'past': 0}
0
2014 can't end fast enough! Stupid dead

@someuser Hate it. Go back to the old format please.
{'future': 0, 'present': 1, 'past': 0}
0
@someuser its a week joke I have with Jessica and my niece:raising_hands:
{'future': 0, 'present': 2, 'past': 0}
0
@someuser You seriously bought a giant TV to play in the box?
{'future': 0, 'present': 0, 'past': 1}
0
Haha who the fuck says that to someone#TrueGentlemen #NOT
{'future': 0, 'present': 1, 'past': 0}
0
If you don't want Hall to eventually ask for a trade then hope for consistency in the message. Change too much you might as well start over
{'future': 1, 'present': 2, 'past': 0}
0
@someuser @someuser wow. That #irony here though, sorry grandpa, we use mac these days #WelcomeToTheWorld #NeverArgueWithFools
{'future': 0, 'present': 2, 'past': 0}
0
@someuser I liked the first one. Never cared enough to watch the sequel(s?). That said, I will only watch the #reboot because of #ChrisPratt
{'future': 1, 'present': 0, 'past': 3}
0
#working.blues#not.enough.vacation.days http://someurl
{'f

{'future': 0, 'present': 1, 'past': 0}
0
Say no more... #bigclub #not http://someurl
{'future': 0, 'present': 0, 'past': 0}
0
@someuser anytime! Cash is acceptable every where. And can be used toward whatever they want/need.
{'future': 1, 'present': 2, 'past': 2}
1
@someuser Nah, it's better we all act like the North Korean govt and police people's private thoughts. #irony
{'future': 0, 'present': 2, 'past': 0}
0
@someuser my 7-year-old got it for me! I think Macy's http://someurl
{'future': 0, 'present': 1, 'past': 1}
1
Blowing your nose so hard your ears pop is the greatest way to start a Wednesday. #not
{'future': 0, 'present': 3, 'past': 0}
0
Today's court proceedings, allowing prisoners access to books in England & Wales, surely confirms it's time to limit judicial review. #irony
{'future': 0, 'present': 3, 'past': 0}
0
Yay I love being awake at 5 in the morning :grinning_face::pistol: #sarcasm
{'future': 0, 'present': 2, 'past': 0}
0
Off to a slow start this morning . :weary_face

Your Jedi mind tricks don't work on me service guy. Glad I dropped the car off for the oil change instead of Av.
{'future': 0, 'present': 1, 'past': 1}
1
I often worry that wealth doesn't have enough influence in society, so pleased to see this being addressed #sarcasm http://someurl
{'future': 0, 'present': 3, 'past': 0}
0
@someuser,@someuser, women say 1 man asltd them, y so long 2 say sumtin? & the young 1s, where the parents? Hotel wit grn ass man? Smh
{'future': 0, 'present': 2, 'past': 1}
1
Some "new" vinyl on the turntable tonight. Here's to you Joe. @someuser #JoeStrummer #Sandinista http://someurl
{'future': 0, 'present': 1, 'past': 1}
1
@someuser who you calling bottle job I'll take teeth. Yeah I'm game mate, at training tonight too, I'll remember your hat #not
{'future': 2, 'present': 2, 'past': 0}
0
To all those who held back in #2014 start again in #2015 as.. "@someuser: It's never too late to start! http://someurl
{'future': 0, 'present': 1, 'past': 1}
1
i'm so funny #not

...

@someuser I thought it funny ;p
{'future': 0, 'present': 1, 'past': 1}
1
The shadowy Investigatory Powers Tribunal says tapping of major Internet cables OK. That's alright then. #Not http://someurl
{'future': 0, 'present': 1, 'past': 0}
0
@someuser Is Jameis Winston getting paid for this photo or is #5 coincidence? #sarcasm
{'future': 0, 'present': 3, 'past': 1}
1
@someuser What do you know about #iubb? Tom Crean is a coaching genius #not!
{'future': 0, 'present': 2, 'past': 0}
0
@someuser @someuser Good points, but Republican party can be just as fractured. Together that makes for a disenchanted electorate.
{'future': 1, 'present': 2, 'past': 1}
1
Greek salad and I are lovers. That is all.
{'future': 0, 'present': 2, 'past': 0}
0
Follow> @someuser and get your FREE DEMO of their industry buzzing app seen at http://someurl
{'future': 0, 'present': 1, 'past': 1}
1
On page 169 of 251 of Harry Potter and the Chamber of Secrets, by J.K. Rowling http://someurl
{'future': 0, 'present': 0, 'p

{'future': 0, 'present': 3, 'past': 0}
0
glad to just find out we had a quiz due yesterday and I didn't even know thanks for the notification blackboard #not
{'future': 0, 'present': 0, 'past': 2}
0
New England (-11.5) over New York Jets; Pittsburgh (-2.5) over Kansas City; Carolina (-4) over Cleveland.
{'future': 0, 'present': 0, 'past': 0}
0
#GuruMantra for #success. #entrepreneur #startup http://someurl
{'future': 0, 'present': 0, 'past': 0}
0
Wow I'm seriously dying here :see-no-evil_monkey::see-no-evil_monkey:
{'future': 0, 'present': 2, 'past': 0}
0
You can have the prettiest face in the world but stretch marks will ruin it all for me.
{'future': 2, 'present': 0, 'past': 0}
0
@someuser no but i followed you and i saw you posted that so i thot i might add on...sorry
{'future': 1, 'present': 3, 'past': 2}
1
@someuser Todd is love, Todd is life #not
{'future': 0, 'present': 3, 'past': 1}
1
Twitter Keeps Insisting I Follow Someone Who I Rather #NOT . . :confused_face:
{'future': 0, '

"I do #not #speak #because I #want to #save #someone. I speak because I like. Who am I #to save... http://someurl
{'future': 0, 'present': 5, 'past': 0}
0
@someuser vodka lime and lemonade is the one :OK_hand::OK_hand::OK_hand:
{'future': 0, 'present': 2, 'past': 0}
0
Sneezing when you have to pee #jusygirlythings #sarcasm
{'future': 0, 'present': 2, 'past': 0}
0
@someuser @someuser "I encourage you... to educated yourself" #irony
{'future': 0, 'present': 3, 'past': 0}
0
D-1 test first day :victory_hand:️:victory_hand:️ #not #read #book #งวย 5555 #doing #home #work =_=
{'future': 0, 'present': 1, 'past': 0}
0
#Celtics RT @someuser: Jameer Nelson will start in his return to Orlando tonight. Marcus Smart will come off the bench.
{'future': 2, 'present': 0, 'past': 0}
0
http://someurl #TeenAnalCasting #What What is the Creampie Crusade?
{'future': 0, 'present': 2, 'past': 0}
0
I love the hypocrisy of the holidays. #NOT
{'future': 0, 'present': 1, 'past': 0}
0
Another day in paradise haha 

Twitter can be so annoying
{'future': 1, 'present': 1, 'past': 0}
0
Being awake since 3AM is always so much fun #sarcasm
{'future': 0, 'present': 2, 'past': 1}
1
@someuser @someuser @someuser You are kidding, right? #irony #poortaste
{'future': 0, 'present': 3, 'past': 0}
0
@someuser I am procrastinating on getting work done b/c I am reading your new book! #Irony
{'future': 0, 'present': 5, 'past': 1}
1
They decided 2 change King's trunks 2 gold, his logos 2 green & his tights to blue. How creative. #Sarcasm.
{'future': 0, 'present': 0, 'past': 1}
0
I'll watch #TakeMeOut like the rest of the country... Hope there's an adorable girl who is ditsy Hahahaha #sarcasm #zzzz
{'future': 1, 'present': 3, 'past': 0}
0
@someuser Presumably you have evidence to back this claim and it's not simply a feeling you have based on your interactions. #Irony
{'future': 0, 'present': 3, 'past': 2}
1
FALSE, slavery was based on economics, @someuser @someuser @someuser @someuser @someuser @1_Free_Man
{'future

0
Fischer ordered fish and chips #irony
{'future': 0, 'present': 0, 'past': 0}
0
#not posting any tweet doesn't mean i don't use twitter !
{'future': 0, 'present': 3, 'past': 0}
0
I messed up today guys. I thought about work. D: #fail #itsabovemypaygradetocare #stillcare #lesigh
{'future': 0, 'present': 1, 'past': 2}
1
"There's a million trillion things I'd rather rather do" profound lyrics big Sean. PROFOUND. #sarcasm #worstsongever
{'future': 1, 'present': 1, 'past': 0}
0
Congratulations @someuser - so well deserved. I hope you enjoy the #irony of this notification from Twitter http://someurl
{'future': 0, 'present': 3, 'past': 1}
1
Sony got hacked again? Oh man, big surprise... #sarcasm: http://someurl
{'future': 0, 'present': 0, 'past': 2}
0
@someuser Gonna miss @someuser, @someuser, @someuser and others, but at least we got Howie Kendrick... #Sarcasm #NotStoked #WTF
{'future': 0, 'present': 0, 'past': 2}
0
@someuser #not gay but I bang that
{'future': 0, 'present': 1, 'past': 0}
0